CSV/Excel 데이터 분석 Agent
- Pandas DataFrame 을 활용하여 분석을 수행하는 Agent 를 생성할 수 있습니다.
- CSV/Excel 데이터로부터 Pandas DataFrame 객체를 생성할 수 있으며, 이를 활용하여 Agent 가 Pandas query 를 생성하여 분석을 수행할 수 있습니다.

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()


True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agent-Toolkits")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agent-Toolkits


In [3]:
import pandas as pd

df = pd.read_csv("./data/titanic.csv")  # CSV 파일을 읽습니다.
# df = pd.read_excel("./data/titanic.xlsx") # 엑셀 파일도 읽을 수 있습니다.
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
!pip install seaborn tabulate

In [7]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import AgentStreamParser
import seaborn as sns

sns.set_style("white")

llm = ChatOpenAI(
    api_key="ollama",
    model="llama3.2:1b",
    base_url="http://localhost:11434/v1",
)

agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=False,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
)

stream_parser = AgentStreamParser()


In [8]:
def ask(query):
    # 질의에 대한 답변을 출력합니다.
    response = agent.stream({"input": query})

    for step in response:
        stream_parser.process_agent_steps(step)

In [9]:
ask("몇 개의 행이 있어?")

[최종 답변]
2 개ของ 행이 있습니다!



1.      0
2.      3

 rows × columns = 2×6=12


In [10]:
ask("남자와 여자의 생존율의 차이는 몇이야?")

[최종 답변]
이 데이터프라amel에 가서 `Survived`라는 칼을 inspect 하면서 그룹화가 된数据를 확인합니다.

```python
import pandas as pd
import numpy as np

#	dataFrame
df = pd.DataFrame({
    'PassengerId': [1, 2, 3, 4, 5],
    'Survived': [0, 1, 1, 0, 0]
})

# 그룹화가 된 데이터를 inspect 합니다.
print(df.groupby('Sex')['Survived'].mean())[['female', 'male']].plot(kind='bar')
```

이 dữ션이는 2グループ을 가꾸고 female와 male의 생존율을 보여줍니다.female은 35/100=0.3의 여자의 생존율, male은 8.05/100=0.0805의 남자의 생존율과 동일합니다.

이러한 데이터가 있습니다.


In [11]:
ask("남자 승객과 여자 승객의 생존율을 구한뒤 barplot 차트로 시각화 해줘")

[최종 답변]
```python
import pandas as pd
import matplotlib.pyplot as plt

#(Data Frame에서 column names에 대한 값으로 전환)
df.columns = ["PassengerId", "Survived", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]

# 남자 승객과 여자 승객의 생존률을 구하고 barplot 차트로 시각화
 survived_male = df["Survived"].where(df["Sex"] == "male").sum()
 survived_female = df["Survived"].where(df["Sex"] != "male").sum()

 plt.figure(figsize=(10,6)) 
 plt.bar(["남자 승객", "หญิง 승객"], [survived_male/survived Female for survived_female in (survived_female/(survived_female+survived_male)): 0.05], color ='lightgreen', label = '남성 승객')
 plt.bar(["남자 승객", "หญิง 승객"], [survived_male/ (survived_male + survived_female) for survived_female in ([survived_female/(survived_female+ survived_male)) ] : 0.05], color ='lightgreen', edgecolor = 'black')   
 plt.xlabel(' 승객 sex ')
 plt.ylabel(' 생존률 (orbits 100%)')
 plt.title('남성 승객과หญิง 승객의 생존률')
 plt.legend()
 plt.xticks(rotation=90, fontsize =10) 
 plt.tight_layout